In [1]:
import urllib
from urllib import request  # urllib.requestモジュールをインポート
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート
import lxml
from newspaper import Article

In [2]:
import datetime as dt 
now = dt.datetime.utcnow()
now -= dt.timedelta(days=1)
year, month, day = now.year, now.month, now.day
limit_date = str(year) + "-" + str(month) + "-" + str(day)
date = str(year) + "-" + str(month)
print(limit_date) 

2021-12-22


In [3]:
website_names = [" - Reuters", " - Bloomberg", " - CNBC", " - TheStreet", " - Fox Business"]
website_names_2 = ["Reuters", "Bloomberg", "CNBC", "TheStreet", "Fox Business"]

In [4]:
def get_items(stock="Apple", date="2021-11-24", homepage="reuters.com"):
    stock = urllib.parse.quote(stock)
    url = "https://news.google.com/rss/search?q=" + stock +  "+after:" + date + "+inurl:" + homepage + "&hl=en-US&gl=US&ceid=US:en"
    response = request.urlopen(url)
    soup = BeautifulSoup(response,"xml")
    response.close()
    elems = soup.find_all("item")
    return elems

In [5]:
with open("./news_site.txt") as f:
    news_sites = f.readlines()
news_sites = list(map(lambda x: x[:-1],news_sites))

In [6]:
with open("./stocks.txt") as f:
    stocks = f.readlines()
stocks = list(map(lambda x: x[:-1],stocks))

In [7]:
import hashlib

In [8]:
articles = []
for news_site, website_name, website_name_2 in zip(news_sites, website_names, website_names_2 ):
    items = get_items(stock="", date=limit_date, homepage=news_site)
    for item in items[:5]:
        title = item.find("title").getText()
        title = str(title.replace(website_name,""))
        link = item.find("link").getText()
        pubdate = item.find("pubDate").getText()
        source = website_name_2
        # data = [title,link,pubdate,source]
        articles.append(
            {
                "title": title,
                "pubDate": pubdate,
                "source": source,
                "link": link,
            }
        )

In [15]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import urllib.parse
import os

In [10]:
sleep_time = 1
try_max_count = 30

In [11]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
}

In [12]:
def get_translated_text(from_lang, to_lang, from_text):
     
    # urlencode
    from_text = urllib.parse.quote(from_text)
     
    #　url作成
    url = 'https://www.deepl.com/translator#' + from_lang +'/' + to_lang + '/' + from_text
     
    #　ヘッドレスモードでブラウザを起動
    options = Options()
    options.add_argument('--headless')
     
    # ブラウザーを起動
    driver = webdriver.Chrome('./chromedriver', options=options)
    driver.get(url)
    driver.implicitly_wait(10)  # 見つからないときは、10秒まで待つ
 
 
    for i in range(try_max_count):
         
        # 指定時間待つ
        time.sleep(sleep_time)  
        html = driver.page_source
        to_text = get_text_from_page_source(html)
         
        try_count = i + 1
     
        if to_text:
            wait_time =  sleep_time * try_count
            print(str(wait_time) + "秒")
             
            # アクセス修了
            break
             
    # ブラウザ停止
    driver.quit()
     
    return to_text


In [13]:
def get_text_from_page_source(html):
    soup = BeautifulSoup(html, features='lxml')
    target_elem = soup.find(class_="lmt__translations_as_text__text_btn")
    text = target_elem.text
     
    return text

In [14]:
titles = ""
for article in articles:
    article = article["title"].replace("/","")
    titles += (article+"\n")
print(titles)

Tencent hands shareholders $16.4 bln windfall in the form of JD.com stake
Ant will be best among China's BAD bunch
EXCLUSIVE Australia puts website accused of fake journalists on register for payment by Facebook, Google
U.S. SEC rejects Apple bid to block shareholder proposal on forced labour -letter
Philippines approves Merck's COVID-19 pill for at-risk patients
Stock Market Today: Dow, S&P Live Updates for Dec. 23, 2021
Omicron vs. Delta: Hospitalization Risk Is Far Lower With New Variant
Eisai Plunges After Alzheimer Drug Fails to Win Japan Backing
A Flotilla of U.S. LNG Cargoes Is Headed to Fuel-Starved Europe
Europe's Power Crunch Shuts Down Factories as Prices Hit Record
Asia-Pacific markets rise; JD shares in Hong Kong plunge 7% after Tencent says it will reduce stake
SEC rejects Apple bid to block shareholder proposal on forced labor, letter shows
Intel apologizes in China over Xinjiang supplier statement
Stock futures rise as market attempts to extend comeback rally to a third

In [22]:
if __name__ == '__main__':
     
    from_lang = 'en'
    to_lang = 'ja'
    #from_text = '提供された翻訳の正確性やサービスの利用可能性について、一切の責任を負いません。'
    from_text = titles
 
    # 翻訳
    to_text = get_translated_text(from_lang, to_lang, from_text)

5秒


In [23]:
japanese_title = list(to_text.split("\n"))
print(japanese_title)

['オミクロン・バリアントがより多くの国で検出され、科学者たちが答えを探している オーストラリア', 'オニキス石炭火力発電所、オランダ政府の支援で閉鎖へ', '石油、ガス、太陽光発電への投資でリビアへの復帰を目指すシェル', 'マッコーリーと仏Engie、Fluenceでオーストラリアのバッテリーを構築', 'アマゾンのクラウド部門、インテルやNvidiaに対抗する新チップを発表', 'Software AG、30億ドル規模のハイテク企業の売却を検討していると言われる', 'Meta (FB)社の役員でDiemの共同開発者であるDavid Marcus氏が会社を去ることになりました。', '南アフリカでオミクロン・バリアントが発生。2歳未満の子供が病院の10％を占める', 'ユーロ圏のインフレ率、予想を上回る4.9％を記録', 'ワクチン競争をリードしてきたPfizer社が30年ぶりに最高の月を迎える', 'Cramer氏、火曜の下落で現金を投入することを提案「売るには遅すぎる', 'オミクロン恐怖症の拡大とパウエルのテーパリング発言でダウは650ポイント下落', 'アジア太平洋地域の株式は、オミクロンのコビットの変動を見守る投資家の影響でほとんどが下落、原油価格は反発', 'S&P 500、BidenがCovidのロックダウンは必要ないと発言したことで、金曜日の暴落から1.3%の反発。', 'サイバーマンデーのオンライン売上高は昨年比1.4％減の107億ドル、史上初のマイナスとなる', '市場の低迷で住宅価格が冷え込む', 'Z世代とミレニアル世代が直面する6つの大きなお金の問題', '今年のサイバーマンデーは購入者に13.9％の負担を強いた', 'バンク・オブ・アメリカ、Squareを中立に、Twitterを買いに変更', 'モデナ、リジェネロン・オミクロンの警告が米国の原油価格を9月の最安値に引きずり込む', 'Twitterの新CEO、Parag Agrawal氏とは？', 'ゴールドマン・サックスのCEOが高額な税金でニューヨーク市に警告', 'セールスフォースのベニオフ氏、共同CEOに就任', '陪審員は万引きで訴えられたWalmartの女性に210万ドルを与える', 'マテル社CEO、バイデン氏がホリデーシーズン中のサプライチェーン危機を解いてくれる

In [24]:
len(japanese_title)

25

In [25]:
for i in range(len(articles)):
    articles[i]["japanese_title"] = japanese_title[i]

In [26]:
articles[0]

{'title': 'Omicron variant detected in more countries as scientists race to find answers Australia',
 'pubDate': 'Sun, 28 Nov 2021 23:02:00 GMT',
 'source': 'Reuters',
 'link': 'https://www.reuters.com/world/new-coronavirus-variant-omicron-keeps-spreading-australia-detects-cases-2021-11-28/',
 'japanese_title': 'オミクロン・バリアントがより多くの国で検出され、科学者たちが答えを探している オーストラリア'}

In [27]:
for article in articles:
    title = article["title"]
    hash_val = hashlib.sha224(title.encode()).hexdigest()
    with open("./articles/importants/articles.txt", "r") as f:
        x = f.read()
    hash_li = list(x.split("\n"))[:-1]
    if hash_val not in hash_li:
        with open("./articles/importants/articles.txt", "a", newline="") as f:
            f.write(hash_val+"\n")
        doc_ref = db.collection("articles").document()
        doc_ref.set({
            u"title": article["title"],
            u"pubDate": article["pubDate"],
            u"link": article["link"],
            u"source": article["source"],
            u"title_ja": article["japanese_title"],  
        })